# Doing inference with SageMaker Built-in Object Detection model

1. [Download the trained model artifact](#download)
1. [Convert training model to deployable model](#convert)
1. [Inference](#inference)
  1. [model load](#load)
  1. [single image inference](#singleinference)
  1. [batch inference](#batchinference)


## setup
### conda_mxnet_p36 is required kernel


In [1]:
import numpy as np
import mxnet as mx
import json
import boto3
import cv2

# Download the trained model artifact <a id='download'></a>

The trained model parameters along with its network definition is stored in a tar.gz file in the output path for the training job. We need to download and unzip it to local disk:

In [2]:
bucket='deeplens-imageclassification-matthew'
endEpoch=60

In [3]:
import pickle
def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)
training_params=load_obj("training_params_kaggle")
job_name=training_params["TrainingJobName"]
print(job_name)

JOB_ID=job_name
S3_OUTPUT_BUCKET =bucket

DEMO-Matthew-kaggle-dog-cat-classification-2020-05-14-14-47-48


In [4]:
sagemaker_client =  boto3.client('sagemaker')
MODEL_ARTIFACT = sagemaker_client.describe_training_job(TrainingJobName=JOB_ID)['ModelArtifacts']['S3ModelArtifacts']
MODEL_ARTIFACT

's3://deeplens-imageclassification-matthew/DEMO-Matthew-kaggle-dog-cat-classification/output/DEMO-Matthew-kaggle-dog-cat-classification-2020-05-14-14-47-48/output/model.tar.gz'

In [5]:
import os 

def make_tmp_folder(folder_name):
    try:
        os.makedirs(folder_name)
    except OSError as e:
        print("{} folder already exists".format(folder_name))

In [6]:
TMP_FOLDER = 'trained-model'
make_tmp_folder(TMP_FOLDER)

!aws s3 cp $MODEL_ARTIFACT $TMP_FOLDER/.

trained-model folder already exists
download: s3://deeplens-imageclassification-matthew/DEMO-Matthew-kaggle-dog-cat-classification/output/DEMO-Matthew-kaggle-dog-cat-classification-2020-05-14-14-47-48/output/model.tar.gz to trained-model/model.tar.gz


Unzipping the model you will find three files in your directory:
```
model_algo_1-symbol.json   <-- neural network definition 
hyperparams.json           <-- hyper parameters  
model_algo_1-0000.params   <-- trained weights for the neural network
```

In [7]:
!tar -xvzf $TMP_FOLDER/model.tar.gz -C $TMP_FOLDER/

image-classification-0027.params
image-classification-0006.params
image-classification-0060.params
image-classification-0015.params
image-classification-0036.params
image-classification-0012.params
image-classification-0033.params
image-classification-symbol.json
image-classification-0018.params
image-classification-0051.params
image-classification-0054.params
image-classification-0057.params
image-classification-0039.params
image-classification-0024.params
image-classification-0009.params
image-classification-0042.params
image-classification-0048.params
image-classification-0021.params
image-classification-0045.params
image-classification-0003.params
model-shapes.json
image-classification-0030.params


When running this script, you need to make sure that command line options you pass in match exactly the hyperparameters of your training job. If you’re unsure, refer the hyperparams.json file in your unpacked model artifacts to confirm. 

In [8]:
!ls -alh $TMP_FOLDER

total 3.5G
drwxrwxr-x 2 ec2-user ec2-user 4.0K May 16 08:03 .
drwxrwxr-x 7 ec2-user ec2-user 4.0K May 16 08:02 ..
-rw-r--r-- 1 ec2-user ec2-user  90M May 16 01:42 image-classification-0000.params
-rw-r--r-- 1 ec2-user ec2-user  90M May 14 14:54 image-classification-0003.params
-rw-r--r-- 1 ec2-user ec2-user  90M May 14 14:55 image-classification-0006.params
-rw-r--r-- 1 ec2-user ec2-user  90M May 14 14:56 image-classification-0009.params
-rw-r--r-- 1 ec2-user ec2-user  90M May 14 14:58 image-classification-0012.params
-rw-r--r-- 1 ec2-user ec2-user  90M May 14 14:59 image-classification-0015.params
-rw-r--r-- 1 ec2-user ec2-user  90M May 14 15:00 image-classification-0018.params
-rw-r--r-- 1 ec2-user ec2-user  90M May 14 15:02 image-classification-0021.params
-rw-r--r-- 1 ec2-user ec2-user  90M May 14 15:03 image-classification-0024.params
-rw-r--r-- 1 ec2-user ec2-user  90M May 14 15:04 image-classification-0027.params
-rw-r--r-- 1 ec2-user ec2-user  90M May 14 15:06 image-classificat